In [7]:
%config InlineBackend.figure_format = 'svg'

(appendix_04)=

# Appendix 4: Regular Expressions

There is a saying that synthetic chemists spend 10% of their time running reactions and 90% of their time purifying compounds. A similar saying could be jokingly said that working with chemical data is 10% performing the intended calculations or analyses on the data and 90% of the time cleaning and organizing the data. While these are both hyperbole, they underline the large amount of effort required to clean materials. This chapter is dedicated to a powerful methods known as **regular expressions**, or **regex** for short, for cleaning and filtering data, especially in situations requiring complex pattern matching. Python [string methods](1.3.3) and [indexing](1.3.2) offer basic search and filtering functionality, but they tend to only allow for identifying simple and consistent patterns. For example, if you want a file name without the file extension (e.g., *titration* from *titration.png*), this can be solved using indexing and the string `split()` method because the file extensions always follow the last period in the full file name. Likewise, parsing data from a [PDB file](16.2.1) can be parsed with only a string search and slicing because PDB files follow very strict formatting rules based on labels and positions in rows. However, not all data follow well-defined formatting rules or there could be more variation that needs to be accounted for. An example that will be addressed below is filting data for Chemical Abstrac Service registration numbers (CAS) numbers which look like 100-53-8 and 172222-30-9 and have have varying lengths.  The good news is that this is a relatively simple task for regular expressions because regular expressions can differentiate between digits, letters, and symbols and provides the ability to seek varying numbers of characters. Regular expressions is not strictly a Python feature but rather is a syntax supported by Python using the `re` module imported below. This module is a built-in Python module, so it comes with every installation of Python.

In [8]:
import re

````{margin}
```{tip}
On the [PubChem](https://pubchem.ncbi.nlm.nih.gov/) site, click Brows Data to see pre-made collections of chemical data such as lists of common solvents, drugs, polymers, etc.
```
````

Below we will first cover some key functions from the `re` modules followed by generating more complex patterns, and finally endind real examples from chemical databases.

## Regular Expression Basics

### `re` Functions

The `re` module provides a series of functions including those listed in **Table 1** that allows the user to search for, split on, or substitute for pattersn within a string. Additional functions can be found on the [Python regular expressions page](https://docs.python.org/3/library/re.html)

**Table 1** Select `re` Functions
| Regex Function | Description |
|---------------|:-------------------|
|`re.findall(pattern, str)` | Returns a list of strings that match the pattern |
|`re.finditer(pattern, str)`| Returns iterable of Match objects |
|`re.search(pattern, str)`  | Returns the *first* pattern match as Match object |
|`re.split(pattern, str)`   | Splits string at pattern matches |
|`re.sub(pattern, replacement, str)`  | Replaces all occurances with new string |

The way these functions work is that the user provides a pattern to search for, which in the most basic scenarios can be a simple string, along with a string in which the function will search for the pattern. In the example below, we search a string of amine names for an aniline derivate by using `'aniline'` as a pattern

In [9]:
amines = '2-methylcyclohexylamine N-methylaniline 3-methylbutylamine N-methyl-3-pentanamine o-methylaniline'
pattern = 'aniline'

re.findall(pattern, amines)

['aniline', 'aniline']

This isn't terribly helpful being that all it tells us is that `'aniline'` appears twice. We can instead use the `re.finditer()` function which returns an iterator providing us with the location of each match using either a `for` loop or `list()` function. We can see below that there are three matches along with the indicies of those matches and the string that matches our pattern.

In [10]:
for x in re.finditer('amine', amines):
    print(x)

<re.Match object; span=(18, 23), match='amine'>
<re.Match object; span=(53, 58), match='amine'>
<re.Match object; span=(76, 81), match='amine'>


In [11]:
list(re.finditer('amine', amines))

[<re.Match object; span=(18, 23), match='amine'>,
 <re.Match object; span=(53, 58), match='amine'>,
 <re.Match object; span=(76, 81), match='amine'>]

We can also use the `re` module to find and replace patterns such as replacing `'aniline'` with `'anilinium'` like below.

In [12]:
re.sub('aniline', 'anilinium', amines)

'2-methylcyclohexylamine N-methylanilinium 3-methylbutylamine N-methyl-3-pentanamine o-methylanilinium'

We could still probably have done the above tasks with string methods and indexing. The real power of regular expressions is its ability to generate more complex and flexible patterns, which is what we address below.

### Symbols & Characters

Let's try something a little more sophisticated by searching for any instance of a methyl *not* located on a nitrogen. This means that the name should have a `'methyl'` string with a hyphenated number before it. The `re` module provides syntax (**Table 2**) for indicating specific types of characters and delimiters. For example, `\d` indicates a digit. Many of these character designators have a negative version using the capital letter, so `\D` signifies any character but a number. 

**Table 2** Regex Character Designators
|Character Type | Present | Not Present | Description/Examples |
|-------|:---------:|:-------------:|----------------------|
|Any character | `.` | |Any character except new line (i.e., \n) |  |
|Digits | \d |\D | Digits from 0-9 |
|Letters/Word characters|  \w | \W | abcABC |
|Space |\s | \S | White space and tabs and end-of-lines |
|Boundary between words | \b | \B | Space, start of line, or non-alphanumeric characters|


Being that we need any number before the methyl, the regex pattern is `\d-methyl`.

|Flag | Description | Example |
|-----|-------------|---------|
|`^` | Beginning of a string |  `^2` finds a 2 at the start of a string  |
|`$` | End of string | `$1` for 1 at end of a string |
|`[]` |Character set to search for | `[.-]` searched for a period or dash |
|`()` | Group - group of patterns to search for | (A\|B\|c) searches for A or B or c |  |


In [13]:
for x in re.finditer(r'\d-methyl', amines):
    print(x)

<re.Match object; span=(0, 8), match='2-methyl'>
<re.Match object; span=(40, 48), match='3-methyl'>


The `\D` could use this as a means of locating methyls that are *not* on a non-aromatic carbon chain because they do not have numbers before them like is done below.

In [14]:
for x in re.finditer(r'\D-methyl', amines):
    print(x)

<re.Match object; span=(24, 32), match='N-methyl'>
<re.Match object; span=(59, 67), match='N-methyl'>
<re.Match object; span=(82, 90), match='o-methyl'>


As another example, below is a string that lists chemical identifiers including chemical names, CAS numbers, and a PubChem CID. The first thing we might want to do is split this up into a list where each item represents a different chemical.

In [15]:
chemicals = ('2-methylphenol   methanol N,N-diethylamine pentanol 281-23-2 ' 
              'ethyl benzoate glycerol 93-89-0 5793 ethanoic acid acetic anhydride')

Using a string method to split based on spaces demonstrated below will not work well because some chemicals (ethyl benzoate, ethanoic acid, and acetic anhydride) have a space in the name. There is also a complication where there are multiple spaces after `'2-methylphenol'`. 

In [16]:
re.split(r'\s', chemicals)

['2-methylphenol',
 '',
 '',
 'methanol',
 'N,N-diethylamine',
 'pentanol',
 '281-23-2',
 'ethyl',
 'benzoate',
 'glycerol',
 '93-89-0',
 '5793',
 'ethanoic',
 'acid',
 'acetic',
 'anhydride']

We will solve this problem below with more tools from regular expressions.

### Quantifiers

Let's first deal with the multiple spaces using quantifiers in **Table 3**. These quantifiers allow us to specify how many of something will be in the pattern. For example, the `a+` will search for one or more a's while `'\s{1,3}'` would look for 1-3 spaces.

**Table 3** Regex Quantifiers
| Flag | Description | Example |
|:----:|-------------|---------|
|{} | Number of preceding character to search for | `\d{3}` for three digits, `\d{3, 7}` for 3-7 digit |
| * | Search for 0 or more | `\w*` for 0 or more letters | 
|+ |Search for 1 or more | `\d+` for one or more digits | 
| ? | 0 or 1 | `\d?` for one or more digits |

Below, we use `\s+` for split our string of chemicals based on one or more spaces.

In [17]:
re.split(r'\s+', chemicals)

['2-methylphenol',
 'methanol',
 'N,N-diethylamine',
 'pentanol',
 '281-23-2',
 'ethyl',
 'benzoate',
 'glycerol',
 '93-89-0',
 '5793',
 'ethanoic',
 'acid',
 'acetic',
 'anhydride']

### Lookahead and Lookbehind


Now let's address the issue of spaces inside the name. IUPAC nomenclature for esters follow the *subsituent carboxylate* pattern where the substituent always ends in *-yl*, and carboxylic acids and anhydrides have *-ic* at the end of the first word (i.e., the carboxyl part). These can be used to identify spaces where the string should *not* be split, and we will carry this out using something known as **lookahead** and **lookbehind** shown in **Table 4**.  These look for the presence or abscence of something before or after our main pattern. We specifically want spaces that do *not* have a *yl* or *ic* preceeding them. Because we do not want a *yl* before our spaces, we will use `(?<!yl)` and place this in front of our `\s+`.

**Table 4** Lookahead and Lookbehind Syntax
|      |Lookahead ($\rightarrow$) | Lookbehind ($\leftarrow$) |
|:----:|:--------:|:----------:|
|Present| `(?=pattern)` | `(?<=pattern)` |
|Abscent| `(?!pattern)` | `(?<!pattern)` |



In [18]:
pattern = r'(?<!yl)\s+'
re.split(pattern, chemicals)

['2-methylphenol',
 'methanol',
 'N,N-diethylamine',
 'pentanol',
 '281-23-2',
 'ethyl benzoate',
 'glycerol',
 '93-89-0',
 '5793',
 'ethanoic',
 'acid',
 'acetic',
 'anhydride']

A lookbehind for the *ic* can also be added like below.

In [19]:
pattern = r'(?<!yl)(?<!ic)\s+'
re.split(pattern, chemicals)

['2-methylphenol',
 'methanol',
 'N,N-diethylamine',
 'pentanol',
 '281-23-2',
 'ethyl benzoate',
 'glycerol',
 '93-89-0',
 '5793',
 'ethanoic acid',
 'acetic anhydride']

In [20]:
pattern = r'(?<!(yl|ic))\s+'
re.split(pattern, chemicals)

['2-methylphenol',
 None,
 'methanol',
 None,
 'N,N-diethylamine',
 None,
 'pentanol',
 None,
 '281-23-2',
 None,
 'ethyl benzoate',
 None,
 'glycerol',
 None,
 '93-89-0',
 None,
 '5793',
 None,
 'ethanoic acid',
 None,
 'acetic anhydride']

### Character Sets

What happens if there are multiple symbols that need to be matched? By placing the symbols or characters to be matched in square brackets, `[]`, anything in the brackets is searched for. For example, it is not uncommon to see numbers separated by either a period *or* dash (e.g., phone numbers), so `[.-]` can be used to indicate that either symbol is a fit. 

Below there is a string of toluene derivatives. If we want to filter for only *para*-substituted toluene derivatives, the name should start with either *p-* or *4-*. Both symbols can be enclosed in the square brackets like `[4p]`. The next challenge is figuring out how to deal with the rest of the symbols. We could try `.+` to indicated any number of more symbols, but this includes whie spaces and returns the rest of the string.

In [21]:
toluene = '3-chlorotoluene 4-methyltoluene p-bromotoluene o-methoxytoluene' 

re.findall(r'[4p]-.+', toluene)

['4-methyltoluene p-bromotoluene o-methoxytoluene']

To solve this, we can again use character sets to include any letter, number, or dash like below. By including the `+` behind the square brackets, this means one or more of these symbols.

In [22]:
re.findall(r'[4p][\d\w-]+', toluene)

['4-methyltoluene', 'p-bromotoluene']

## Finding CAS Numbers

Let's now do some extra examples. When downloading data files from [PubChem](https://pubchem.ncbi.nlm.nih.gov/), the CAS number is mixed in with other names and numberical identifiers. There are two challenges here. The first is that CAS numbers vary in length. They are always three segments of numbers separated by hyphens, such as 58-08-2 or 2501-94-2, where the second segment is always two digits and the thirds is always a single digit. However the first segment varies from 2-7 digits. The second major issue is that the CAS numbers are mixed in with other chemical identifiers such as CID numbers, common names, and IUPAC names. These other identifiers can include hyphens and numbers, so indexing and string searches cannot easily filter for CAS numbers without a long series of boolean conditions. 

This is a relatively simple task for regular expressions. We indicate digits with the `\d` and use curly brackets to indicate the number of digits as demonstrated below.

In [23]:
re.findall(r'\d{2,7}-\d{2}-\d', chemicals)

['281-23-2', '93-89-0']

PubChem allows for the download of datasets that include a *Synonym* column which includes

## Parse NMR Data

When data on an NMR spectrum is reported in the literature, it follows relatively strict formatting rules, but these rules are designed to be ready by humans, not machines. To make things more complicated, there are numerous commas and spaces in the data making it difficult to use these as delimiters, so regular expressions are ideal for parsing this kind of data. Below is the $^1$H NMR data for butanamide in DMSO-$d_6$ following [Americal Chemical Society guidelines](https://researcher-resources.acs.org/publish/data_guidelines).

> $^1$H NMR ((CD)$_3$SO, 400 MHz): $\delta$ 7.23 (br, 1H), 6.70 (br, 1H), 2.00 (t, 2H, J = 7.3 Hz), 1.48 (tq, 2H, J = 7.3, 7.3 Hz), 0.84 (t, 3H, J = 7.3 Hz).

As an example, we will extract the entries for each signal in the NMR spectrum. Each entry looks like `7.23 (br, 1H)` or `0.84 (t, 3H, J = 7.3 Hz)` where the decimal is the chemical shift and additional information on the signal is provided in the parentheses behind the chemical shift.

In [80]:
proton = ('1H NMR ((CD)3SO, 400 MHz): δ 7.23 (br, 1H), 6.70 (br, 1H),' 
          '2.00 (t, 2H, J = 7.3 Hz), 1.48 (tq, 2H, J = 7.3, 7.3 Hz),' 
          '0.84 (t, 3H, J = 7.3 Hz).')

Each signal starts with a number to two decimal places, but there may be one or two digits before the decimal place. Even though our example always has one digit before the decimal, we want our code to be rebust and versitile. The regular expression for this number is `'\d{1,2}.\d{2}'`.

In [76]:
nmr_pattern = r'\d{1,2}.\d{2}'
re.findall(nmr_pattern, proton)

['7.23', '6.70', '2.00', '1.48', '0.84']

Next, the informatou about the signal is stored in parentheses separated from the chemical shift by a space. We will use `'\s+'` just in case someone accidentially used multiple spaces. Because parentheses are a regular expression character, we need to preceed it with a backslash to indicate that we actually mean just a parenthese character.

In [86]:
nmr_pattern = r'\d+.\d{2}\s+\('
re.findall(nmr_pattern, proton)

['7.23 (', '6.70 (', '2.00 (', '1.48 (', '0.84 (']

Inside the parentheses is the 
- Splitting pattern as one or more letters, `'\w+'`
- Integration as an integer with an `H`, so `'\d+H'`
- Coupling information as starting with `J =` followed by a number to two decimal places, so `'J\s+=\s+\d+.\d+\s+Hz'`. 

In [88]:
nmr_pattern = r'\d+.\d{2}\s+\(\w+,\s+\d+H,\s+J\s+=\s+\d+.\d\s+Hz\)'

re.findall(nmr_pattern, proton)

['2.00 (t, 2H, J = 7.3 Hz)', '0.84 (t, 3H, J = 7.3 Hz)']

The current pattern misses the signals that do not include the coupling information or have multiple coupling constants. This is where quantifier are helpful. By placing the regular expression that pattern matches `, J = 7.3` in square brackets followed by an asterisk like below, it indicates that there could be zero or more of these.

    [,\s+J\s?=\s?\d+.\d]*


In [102]:
nmr_pattern = r'\d+.\d{2}\s\(\w+,\s+\d+H[,\s+J\s?=\s?\d+.\d]*\sHz\)'

re.findall(nmr_pattern, proton)

['2.00 (t, 2H, J = 7.3 Hz)',
 '1.48 (tq, 2H, J = 7.3, 7.3 Hz)',
 '0.84 (t, 3H, J = 7.3 Hz)']

Now the regular expression finds all signals that have coupling constants but is still missing the two without coupling constants. This is because the pattern still requires a `' Hz'`. Because there should be either zero or one of these, the regular expression that searches for this should also be enclosed in square brackets and followed by a `*` like below

    [\sHz]*

In [105]:
nmr_pattern = r'\d+.\d{2}\s\(\w+,\s+\d+H[,\s+J\s?=\s?\d+.\d]*[\sHz]*\)'

re.findall(nmr_pattern, proton)

['7.23 (br, 1H)',
 '6.70 (br, 1H)',
 '2.00 (t, 2H, J = 7.3 Hz)',
 '1.48 (tq, 2H, J = 7.3, 7.3 Hz)',
 '0.84 (t, 3H, J = 7.3 Hz)']

It looks like find all the singals. One more addition that would be helpful in making the code more robust is to add the possibility of a negative chemical shift. While these values are typically positive, negative values do show up in situations such as silanes with Si-H bonds or metal hydrides. To allow for this possibiliyt, a `-?` is placed in the front indicated that the negative may or may not be there. To test this, an extra negative resonance was added just for testing purposes.

In [109]:
proton = ('1H NMR ((CD)3SO, 400 MHz): δ 7.23 (br, 1H), 6.70 (br, 1H),' 
          '2.00 (t, 2H, J = 7.3 Hz), 1.48 (tq, 2H, J = 7.3, 7.3 Hz),' 
          '0.84 (t, 3H, J = 7.3 Hz), -0.54 (s, 1H).')

In [110]:
nmr_pattern = r'-?\d+.\d{2}\s\(\w+,\s+\d+H[,\s+J\s?=\s?\d+.\d]*[\sHz]*\)'
re.findall(nmr_pattern, proton)

['7.23 (br, 1H)',
 '6.70 (br, 1H)',
 '2.00 (t, 2H, J = 7.3 Hz)',
 '1.48 (tq, 2H, J = 7.3, 7.3 Hz)',
 '0.84 (t, 3H, J = 7.3 Hz)',
 '-0.54 (s, 1H)']

If someone wanted to extract values from the NMR signals, additional regular expressions could be written to iterate through the list and extract the desired information.

## Further Reading

1. Documentation for `re` package. [https://docs.python.org/3/library/re.html](https://docs.python.org/3/library/re.html) (free resource)

   > Python `re` module documentation. Provides a good list of flags.
   
3. Datacampe Regular Expressions Cheat Sheet. [https://www.datacamp.com/cheat-sheet/regular-expresso](https://www.datacamp.com/cheat-sheet/regular-expresso) (free resource)

   > One-page summary of key regular expression pattern characters good for hanging above a desk.

5. 